## Impact of the 2014 FIFA World Cup on Host States in Brazil

This notebook investigates the impact of hosting the 2014 FIFA World Cup on Brazil's states. The analysis focuses on key economic and demographic outcomes using state-level data. Below are the key aspects of the notebook:

- **Objective**:  
  - Explore the economic and demographic impacts of the 2014 FIFA World Cup at the state level.  
  - Examine whether hosting the World Cup delivered measurable benefits or harms to host states.  

- **Methods Used**:  
  - **Difference-in-Differences (DiD)**: Estimates the changes in key outcomes by comparing trends in host states to non-host states before and after the World Cup.  
  - **Synthetic Control Methods (SCM)**: Constructs a synthetic version of each host state using non-host states with similar trends before the world cup to estimate what might have occurred without hosting.  

- **Key Variables**:  
  - Population growth.  
  - Economic development metrics (e.g., GDP, employment rates).  
  <!-- - Socio-economic indicators (e.g., infrastructure spending, tourism).   -->

- **Purpose**:  
  - Quantify the causal effects of hosting the World Cup.  
  - Assess the trade-offs between the benefits of hosting and the associated financial burden.  

<!-- - **Contribution**:  
  - Provides insights into the long-term impacts of mega-events on regional economies.  
  - Informs debates on whether hosting international sporting events justifies the investment.   -->


In [49]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [50]:
population_df = pd.read_excel('/Users/muhammadmuhdhar/Desktop/Repo/WorldCupHostCities_Impact/data/population_data.xlsx')
population_df.head()

,SEXO,CÓD.,SIGLA,LOCAL,2000,2001,2002,2003,2004,2005,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Ambos,0,BR,Brasil,3423475,3347313,3274356,3212295,3163041,3123546,...,3012970,2948637,2883226,2920425,2885318,2777978,2688491,2597799,2527241,2484344
1,Ambos,0,BR,Brasil,3450022,3406966,3332612,3261091,3200484,3152623,...,2943154,3002376,2937001,2870232,2910089,2879163,2772134,2680031,2587620,2519770
2,Ambos,0,BR,Brasil,3461038,3444450,3401900,3327924,3256791,3196543,...,2907915,2936667,2995070,2928703,2863990,2906676,2876276,2767175,2673622,2583366
3,Ambos,0,BR,Brasil,3469109,3458052,3441638,3399284,3325501,3254582,...,2916787,2903600,2931742,2989393,2924609,2862071,2904924,2873055,2762809,2670309
4,Ambos,0,BR,Brasil,3477903,3466901,3455987,3439662,3397467,3323811,...,2898606,2913739,2900133,2927764,2986627,2923442,2861079,2902777,2869950,2760203


In [51]:
# Define the range of years for analysis
years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 
         2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 
         2020, 2021, 2022, 2023, 2024]

# List of Brazilian states for filtering the data
states = [
    "Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará",
    "Distrito Federal", "Espírito Santo", "Goiás", "Maranhão",
    "Mato Grosso", "Mato Grosso do Sul", "Minas Gerais", "Paraná",
    "Paraíba", "Pará", "Pernambuco", "Piauí", "Rio Grande do Norte",
    "Rio Grande do Sul", "Rio de Janeiro", "Rondônia", "Roraima",
    "Santa Catarina", "Sergipe", "São Paulo", "Tocantins"
]

# Group population data by 'LOCAL' (state) and sum across the specified years
population_df = population_df.groupby('LOCAL', as_index=False)[years].sum()

# Filter the dataset to include only states
population_df = population_df[population_df['LOCAL'].isin(states)]

# Calculate percentage growth in population over time for each state
# Drop the 'LOCAL' column temporarily for percentage change calculations
population_growth_df = population_df.drop('LOCAL', axis=1)

# Calculate percentage change (growth rate) across years and multiply by 100 to get percentages
population_growth_df = population_growth_df.pct_change(axis=1) * 100

# Reinsert the 'STATE' column for easier reference after calculations
population_growth_df.insert(0, "STATE", population_df['LOCAL'])

# Remove the year 2000 since percentage growth cannot be calculated for the first year
population_growth_df.drop(2000, axis=1, inplace=True)

# Reshape the DataFrame from wide to long format for better analysis and visualization
population_growth_long_df = population_growth_df.melt(id_vars=['STATE'], var_name='YEAR', value_name='Population_growth')
population_growth_long_df['YEAR'] = population_growth_long_df['YEAR'].astype(int)

population_growth_long_df.head()

,STATE,YEAR,Population_growth
0,Acre,2001,3.064908
1,Alagoas,2001,1.247077
2,Amapá,2001,4.333029
3,Amazonas,2001,2.477657
4,Bahia,2001,0.901883


In [52]:
gdp = pd.read_excel('/Users/muhammadmuhdhar/Desktop/Repo/WorldCupHostCities_Impact/data/GDP.xlsx')
gdp = gdp.rename(columns={"Unnamed: 0": "STATE"})

# Reshape the GDP DataFrame from wide to long format for better analysis and visualization
gdp_long_df = gdp.melt(id_vars=['STATE'], var_name='YEAR', value_name='GDP')
gdp_long_df['YEAR'] = gdp_long_df['YEAR'].astype(int)

gdp_long_df.head()

,STATE,YEAR,GDP
0,Rondônia,2002,7467.629889
1,Acre,2002,2971.301278
2,Amazonas,2002,22093.338008
3,Roraima,2002,2392.032670
4,Pará,2002,26482.158869


In [53]:

print(population_growth_long_df.shape)
print(gdp_long_df.shape)

(648, 3)
(567, 3)


In [54]:
print("\nMissing Values per Column:")
missing_values = population_growth_long_df.isnull().sum()
missing_values[missing_values > 0]


Missing Values per Column:


Series([], dtype: int64)

In [55]:
print("\nMissing Values per Column:")
missing_values = gdp_long_df.isnull().sum()
missing_values[missing_values > 0]


Missing Values per Column:


Series([], dtype: int64)

## Difference-in-Differences (DiD) Analysis

### Objective
The Difference-in-Differences (DiD) analysis aims to evaluate the causal impact of hosting the 2014 FIFA World Cup on key outcomes, such as GDP growth, by comparing trends in host states (treatment group) and non-host states (control group) before and after the event. 

### Methodology
1. **Groups**:
   - **Treatment Group**: States that hosted the World Cup matches (e.g., São Paulo, Rio de Janeiro, Amazonas).
   - **Control Group**: States that did not host any matches, serving as a baseline for comparison.

2. **Time Periods**:
   - **Pre-Treatment Period**: Years before 2014.
   - **Post-Treatment Period**: Years after 2014, including and beyond the treatment year.

3. **Key Variables**:
   - **Dependent Variable**: `Population Growth (%)`, `GDP` – the year-over-year percentage change in GDP.
   - **Independent Variables**:
     - `post_treatment`: A binary indicator (1 for years after 2014, 0 otherwise).
     - `treatment_group`: A binary indicator (1 for host states, 0 otherwise).
     - Interaction Term (`post_treatment * treatment_group`): Captures the differential effect of the World Cup on host states in the post-treatment period.

In [56]:
host_states = [
    "São Paulo","Rio de Janeiro","Amazonas","Distrito Federal",
    "Bahia","Ceará","Pernambuco","Rio Grande do Sul",
    "Paraná","Rio Grande do Norte","Mato Grosso","Minas Gerais"    
]

non_host_states = [
    "Acre","Alagoas","Amapá",           
    "Maranhão","Mato Grosso do Sul","Espírito Santo",  
    "Goiás","Paraíba","Pará","Piauí",           
    "Rondônia","Roraima","Santa Catarina",  
    "Sergipe","Tocantins"        
]

In [57]:
# Define the treatment year as a Pandas Period object with annual frequency
# This represents the year when the World Cup took place
world_cup_treatment = pd.Period("2014", freq="Y")

# Convert the 'YEAR' column in the DataFrame to a Pandas PeriodIndex with annual frequency
# Ensures that the 'YEAR' column is treated as a time-based index, useful for comparisons
population_growth_long_df['YEAR'] = pd.PeriodIndex(population_growth_long_df['YEAR'].astype(str), freq='Y')

# Create a new column 'post_treatment' indicating whether each year is after the treatment year
# Assigns 1 if the year is after 2014, and 0 otherwise
population_growth_long_df['post_treatment'] = (population_growth_long_df['YEAR'] > world_cup_treatment).astype(int)

# Create a new column 'treatment_group' to indicate whether each state is a host state
# Assigns 1 for host states and 0 for non-host states
population_growth_long_df['treatment_group'] = (population_growth_long_df['STATE'].isin(host_states)).astype(int)

population_growth_long_df.sample(10)


,STATE,YEAR,Population_growth,post_treatment,treatment_group
230,Paraíba,2009,0.767676,0,0
149,Paraíba,2006,0.872991,0,0
399,Rondônia,2015,0.910058,1,0
21,Rondônia,2001,1.503067,0,0
555,Pará,2021,0.649259,1,0
543,Amazonas,2021,0.941152,1,1
145,Mato Grosso,2006,1.657419,0,1
455,Santa Catarina,2017,1.768330,1,0
351,Acre,2014,1.454024,0,0
525,Minas Gerais,2020,0.551032,1,1


In [60]:
formula = 'Population_growth ~ post_treatment + treatment_group + post_treatment*treatment_group'

model = smf.ols(formula, data=population_growth_long_df)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:      Population_growth   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.194
Method:                 Least Squares   F-statistic:                     53.04
Date:                Sat, 30 Nov 2024   Prob (F-statistic):           1.20e-30
Time:                        23:27:03   Log-Likelihood:                -657.92
No. Observations:                 648   AIC:                             1324.
Df Residuals:                     644   BIC:                             1342.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [70]:
# Convert the 'YEAR' column into a Pandas PeriodIndex with an annual frequency
# This ensures the 'YEAR' column is treated as time-based data, enabling proper time comparisons
gdp_long_df['YEAR'] = pd.PeriodIndex(gdp_long_df['YEAR'].astype(str), freq='Y')

# Create a new column 'post_treatment' to indicate whether the year is after the treatment year (2014)
# Assigns 1 if the year is after 2014 (post-treatment), and 0 otherwise
gdp_long_df['post_treatment'] = (gdp_long_df['YEAR'] > world_cup_treatment).astype(int)

# Create a new column 'treatment_group' to indicate whether the state is in the treatment group (host states)
# Assigns 1 for host states and 0 for non-host states
gdp_long_df['treatment_group'] = (gdp_long_df['STATE'].isin(host_states)).astype(int)

# Ensure the 'treatment_group' column is correctly assigned (this duplicates the assignment above, likely redundant)
# Assigns 1 for states in the host states list, and 0 for all others
gdp_long_df['treatment_group'] = (gdp_long_df['STATE'].isin(host_states)).astype(int)


In [71]:
gdpformula = 'GDP ~ post_treatment + treatment_group + post_treatment*treatment_group'

gdpmodel = smf.ols(gdpformula, data=gdp_long_df)
results = gdpmodel.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    GDP   R-squared:                       0.241
Model:                            OLS   Adj. R-squared:                  0.237
Method:                 Least Squares   F-statistic:                     59.49
Date:                Sat, 30 Nov 2024   Prob (F-statistic):           2.04e-33
Time:                        23:31:57   Log-Likelihood:                -7960.0
No. Observations:                 567   AIC:                         1.593e+04
Df Residuals:                     563   BIC:                         1.595e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [19]:
years_before_worldcup = [2001,2002,2003,2004,2005,2006,
                         2007,2008,2009,2010,2011,2012,2013]

population_growth_before_worldcup = population_growth_df[years_before_worldcup]
population_growth_before_worldcup.insert(0, "STATE", population_df['LOCAL'])

population_growth_before_worldcup.head()

,STATE,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,Acre,3.064908,3.008291,2.949945,2.880528,2.802299,2.716190,2.604500,2.525530,2.326323,1.911759,1.744004,1.711027,1.535593
1,Alagoas,1.247077,1.126324,1.025180,0.944784,0.871786,0.790968,0.680911,0.637000,0.569046,0.428154,0.336694,0.322962,0.328393
2,Amapá,4.333029,4.023548,3.752784,3.520077,3.306711,3.009246,2.802095,2.641972,2.537367,2.175927,1.860734,1.723138,1.686222
3,Amazonas,2.477657,2.378456,2.307652,2.261516,2.233362,2.230557,2.198126,2.156981,2.085166,1.886182,1.709537,1.695615,1.691736
4,Bahia,0.901883,0.848185,0.803037,0.769127,0.739972,0.688512,0.641239,0.623928,0.602399,0.465607,0.351971,0.389093,0.407015


In [20]:
# Reshape the DataFrame from wide to long format
long_df = population_growth_before_worldcup.melt(id_vars=['STATE'], var_name='YEAR', value_name='POPULATIONGROWTH')
long_df['YEAR'] = long_df['YEAR'].astype(int)  # Convert years to integers

# Separate host and non-host states
host_states_population_before_worldcup = long_df[long_df['STATE'].isin(host_states)]
non_host_states_population_before_worldcup = long_df[long_df['STATE'].isin(non_host_states)]

print(host_states_population_before_worldcup.shape)
print(non_host_states_population_before_worldcup.shape)

(156, 3)
(195, 3)


In [21]:
for idx, host_state in enumerate(host_states_population_before_worldcup['STATE'].unique()):
    print(idx, host_state)

0 Amazonas
1 Bahia
2 Ceará
3 Distrito Federal
4 Mato Grosso
5 Minas Gerais
6 Paraná
7 Pernambuco
8 Rio Grande do Norte
9 Rio Grande do Sul
10 Rio de Janeiro
11 São Paulo


In [22]:

# Calculate rows dynamically
rows = (num_host_states + 2) // 3

fig, axes = plt.subplots(rows, 3, figsize=(18, 6 * rows), sharey=True)

# Flatten axes to a 1D array for consistent indexing
axes = np.array(axes).flatten()

# Keep track of handles and labels for a combined legend
handles = []
labels = []

for id, state in enumerate(host_states_population_before_worldcup['STATE'].unique()):
    ax = axes[id]
    
    # Filter data for the specific host state
    host_data = host_states_population_before_worldcup[host_states_population_before_worldcup['STATE'] == state]
    
    # Plot host state population growth
    host_line, = ax.plot(host_data['YEAR'], host_data['POPULATIONGROWTH'], 
                         linewidth=2, label=f'Host: {state}')
    
    # Add to legend tracking (only add once per type)
    if not any(lbl == host_line.get_label() for lbl in labels):
        handles.append(host_line)
        labels.append(host_line.get_label())
    
    # Plot non-host states for comparison
    for non_host_state in non_host_states_population_before_worldcup['STATE'].unique():
        non_host_data = non_host_states_population_before_worldcup[non_host_states_population_before_worldcup['STATE'] == non_host_state]
        non_host_line, = ax.plot(non_host_data['YEAR'], non_host_data['POPULATIONGROWTH'], 
                                 linestyle='--', alpha=0.6, label=f'Non-host: {non_host_state}')
        
        # Add to legend tracking (only add once per type)
        if not any(lbl == non_host_line.get_label() for lbl in labels):
            handles.append(non_host_line)
            labels.append(non_host_line.get_label())
    
    # Set title and labels for clarity
    ax.set_title(state)
    ax.set_xlabel("Year")
    ax.set_ylabel("Population Growth")
    
# Hide unused subplots
for id in range(len(host_states_population_before_worldcup['STATE'].unique()), len(axes)):
    axes[id].axis('off')

# Create a single legend outside the subplots
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3, fontsize="small")

plt.tight_layout()
plt.show()


NameError: name 'num_host_states' is not defined